In [1]:
import numpy as np
import random
from numpy import linalg
import matplotlib.pyplot as plt

A method to generate a series of times (in days) that would reflect realistic sample times from the Vera Rubin observatory given a duration of the gap in terms of days and a duration in terms of years

In [2]:
def generate_spaced_times(space_duration, total_days):
    # Generate a list of times that are approximately the given days apart
    times_clean = np.linspace(0, total_days, int(total_days / space_duration))
    variability = np.random.normal(0, 1, len(times_clean) - 1)
    variability = np.insert(variability, 0, 0)
    times = np.add(times_clean, variability)
    
    return times


def generate_gap(times, gap_duration):
    # Generate a random location each year at which the gap will take place
    gap_beginning = random.randint(0, 365)
    gap_end = gap_beginning + gap_duration
    
    # Generate a list that will be the values that each gap begins and then ends
    intervals = [[gap_beginning, gap_end]]
    while intervals[len(intervals) - 1][0] + 365 < np.max(times):
        intervals.append([intervals[len(intervals) - 1][0] + 365, intervals[len(intervals) - 1][1] + 365])
        
    for interval in intervals:
        times = [time for time in times if (interval[0] >= time <= interval[1]) or (interval[0] <= time >= interval[1])]
    
    return times


def generate_Vera_Rubin_times(gap_duration, total_duration):
    total_days = total_duration * 365 # The amount of days the light curve will be
    
    # Generate a list of times that are approximately 3 days apart
    times = generate_spaced_times(3, total_days)
    
    # Delete the values in a random location of the times for a given gap
    times_gapped = generate_gap(times, gap_duration)
    times_gapped = sorted(times_gapped, key = lambda x:float(x)) 
    
    return times_gapped

A method to generate a DRW light curve

In [3]:
def generate_DRW_LC(t_rest, sigma, tau, z):
  # Account for redshift:
  t_obs = []
  
  # Account for redshift
  tau_rf = tau / (1 + z)

  # Make first value of the light curve and declare a light curve list
  lc1 = np.random.normal(0, sigma, None)
  lc = [lc1]

  # Fill the lightcurve list
  for i in range(1, len(t_rest)):
    dt = t_rest[i] - t_rest[i - 1]
    lc_next = lc[i - 1] * np.exp(-dt / tau_rf) + np.random.normal(0, (sigma) * (1 - np.exp((-2 * dt) / tau_rf)), None)
    lc.append(lc_next)
  
  return lc

A method to fit DRWs to sigma and taus. We also have methods helper methods here

In [4]:
def fit_DRW(t_rest, lc, z, steps):
    best = None, None
    loglikelihood = None
    best_loglikelihood = None
    for t in range(steps + 1):
        for s in range(steps + 1):
            sigma, tau = .1 + (s / steps) * 10, 1 + (t / steps) * 100
            loglikelihood = get_loglikelihood(t_rest, lc, sigma, tau, 0)
            if best_loglikelihood == None or loglikelihood > best_loglikelihood:
                best_loglikelihood = loglikelihood
                best = sigma, tau
  
    return best

def get_loglikelihood(t_rest, lc, sigma, tau, z):
    S = generate_cov_matrix(t_rest, sigma, tau) # Cov matrix of the lc
    C_inv = linalg.inv(S) # Inverse of C (C = S + N) N is cov matrix of noise
    L = np.zeros((len(t_rest), 1)) + 0.5 # Matrix of 0.5s
    L_trans = np.transpose(L) # L transposed
    y = np.zeros((len(t_rest), 1)) # Matrix of given light curve
    y[:,0] = lc
    y_trans = np.transpose(y) # y transposed
    C_inv_up1 = C_inv 
    denom = np.matmul(np.matmul(L_trans, C_inv), L) # Denominator of likelihood equation
    A = np.matmul(L_trans, C_inv)
    B = np.matmul(C_inv,L)
    C_inv_uptack = C_inv_up1 - np.matmul(B,A) / denom
    expval = -1.0 * (np.matmul(np.matmul(y_trans, C_inv_uptack), y)) / 2
    loglikelihood = np.log(pow(linalg.det(S), -1/2)) + np.log(pow(np.abs(denom), -1/2)) + expval 
 
    return loglikelihood[0][0]

def generate_cov_matrix(t_rest, sigma, tau):
    # Create a square matrix with # of columns and rows = len(t_rest)
    matrix = np.zeros((len(t_rest),len(t_rest)))

    # Fill in the matrix with equation describing the covariance
    for i in range(len(t_rest)):
        for j in range(len(t_rest)):
            delta_t = abs(t_rest[i] - t_rest[j])
            matrix[i][j] = sigma * sigma * np.exp(-1.0 * delta_t / tau) 

    return matrix

Generate and fit a DRW

In [5]:
sigma, tau = 2.0, 10
times_VR = generate_Vera_Rubin_times(100, 10)

lc = generate_DRW_LC(times_VR, sigma, tau, 0)
sigma_est, tau_est = fit_DRW(times_VR, lc, 0, 35)
print('Sigmas: ', sigma, sigma_est)
print('Taus: ', tau, tau_est)

plt.plot(times_VR, lc)
plt.plot(times_VR, generate_DRW_LC(times_VR, sigma_est, tau_est, 2), 'r')
plt.title("DRW Fit")
plt.show

/Users/adamboesky/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
/Users/adamboesky/opt/anaconda3/lib/python3.7/site-packages/numpy/linalg/linalg.py:2116: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/adamboesky/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log


KeyboardInterrupt: 